<a href="https://colab.research.google.com/github/JayThibs/huggingface-course-cer-workshop/blob/main/notebooks/01_labeling_VCs_with_zero_shot_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Labeling VCs with Zero-Shot Learning using Huggingface Transformers

# Installations

In [1]:
!pip install transformers --quiet
!pip install tqdm --quiet

     |████████████████████████████████| 3.3 MB 4.2 MB/s 
     |████████████████████████████████| 596 kB 71.2 MB/s 
     |████████████████████████████████| 3.3 MB 61.0 MB/s 
     |████████████████████████████████| 895 kB 60.0 MB/s 
     |████████████████████████████████| 61 kB 612 kB/s 


# Imports

In [2]:
import os
import requests
import pandas as pd
from transformers import pipeline
from bs4 import BeautifulSoup
import nltk
from nltk.stem.porter import *
from tqdm import tqdm

nltk.download('stopwords', quiet=True)

True

# Loading the Data

In [3]:
csv_url = 'https://raw.githubusercontent.com/JayThibs/huggingface-course-cer-workshop/main/data/zero_shot_esa_index_train_128_max_tokens.csv'
df_joined = pd.read_csv(csv_url, index_col=0)

In [4]:
len(df_joined)

28891

In [5]:
df_joined.head()

,Index,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,text,label
0,9134,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Co...",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,14,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-wor...,1059614_14_lattice-v_1.csv,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABO...,-1
1,9135,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Co...",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,17,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_17_lattice-v_1.csv,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,-1
2,9136,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Co...",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,18,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_18_lattice-v_1.csv,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,-1
3,9137,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Co...",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,19,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_19_lattice-v_1.csv,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,-1
4,9138,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,1059614,https://apps.cer-rec

In [6]:
# Explicitly ask for tensor allocation on CUDA device :0
classifier = pipeline("zero-shot-classification", device=0)

No model was supplied, defaulted to facebook/bart-large-mnli (https://huggingface.co/facebook/bart-large-mnli)


Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [7]:
df_joined.iloc[100]['text']

'Table 425 Major Fire Departments in the Infrastructure and Services in the RAA FulltimeEmployees Equipment ServicesProvided AreasCommunitiesServed HudsonsHope FireDepartment 1 fire chief 25 firefighters Not specified fireprevention andinspectionservices City of Hudsons Hopeand surrounding area MoberlyLake FireDepartment 1 fire chief Severalfirefighters notspecified Not specified Not specified Moberly Lake areaWest Moberly FirstNation and SaulteauFirst Nation SOURCE Modified from PRRD 2013a '

In [8]:
sequences = []

for x in range(len(df_joined)):
  sequences.append(df_joined.iloc[x]['text'][0:250])

In [9]:
len(sequences)

28891

In [10]:
sequences[0]

'TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABORIGINAL FIELD STUDY PARTICIPATION FOR THE PROJECT Survey Date Aboriginal Communities Detail July 9 to 12 2011 Blueberry River First Nation Halfway River First Nation McLeod Lake Indian Band North East Mti'

In [11]:
candidate_labels = """Physical and meteorological environment,Soil and soil productivity,Vegetation,Water quality and quantity,Fish and fish habitat,Wetlands,Wildlife and wildlife habitat,Species at Risk or Species of Special Status and related habitat,Greenhouse gas (GHG) emissions and climate change,GHG Emissions and Climate Change – Assessment of Upstream GHG Emissions,Air emissions,Acoustic environment,Electromagnetism and Corona Discharge,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure and services,Employment and economy,Environmental Obligations,Rights of Indigenous Peoples""".split(',')
hypothesis_template = "The data from this table is about {}."
candidate_labels

['Physical and meteorological environment',
 'Soil and soil productivity',
 'Vegetation',
 'Water quality and quantity',
 'Fish and fish habitat',
 'Wetlands',
 'Wildlife and wildlife habitat',
 'Species at Risk or Species of Special Status and related habitat',
 'Greenhouse gas (GHG) emissions and climate change',
 'GHG Emissions and Climate Change – Assessment of Upstream GHG Emissions',
 'Air emissions',
 'Acoustic environment',
 'Electromagnetism and Corona Discharge',
 'Human occupancy and resource use',
 'Heritage resources',
 'Navigation and navigation safety',
 'Traditional land and resource use',
 'Social and cultural well-being',
 'Human health and aesthetics',
 'Infrastructure and services',
 'Employment and economy',
 'Environmental Obligations',
 'Rights of Indigenous Peoples']

In [12]:
import gc
import torch

gc.collect()

torch.cuda.empty_cache()

In [13]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |    1553 MB |    1553 MB |    1553 MB |       0 B  |\n|       from large pool |    1552 MB |    1552 MB |    1552 MB |       0 B  |\n|       from small pool |       1 MB |       1 MB |       1 MB |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |    1553 MB |    1553 MB |    1553 MB |       0 B  |\n|       from large pool |    1552 MB |    1552 MB |

In [14]:
from itertools import zip_longest
 
def grouper(iterable_obj, count, fillvalue=None):
    args = [iter(iterable_obj)] * count
    return zip_longest(*args, fillvalue=fillvalue)

output = [] 

for sequence_batch in tqdm(grouper(sequences, 2, "")):
    # print(sequence_batch)
    output.append(classifier(sequence_batch, candidate_labels, hypothesis_template=hypothesis_template, multi_label=True))

10it [00:08,  1.18it/s]/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:1079: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
14446it [3:01:43,  1.32it/s]


In [16]:
output[0]

[{'labels': ['Water quality and quantity',
   'Traditional land and resource use',
   'Environmental Obligations',
   'Rights of Indigenous Peoples',
   'Heritage resources',
   'Social and cultural well-being',
   'Infrastructure and services',
   'Species at Risk or Species of Special Status and related habitat',
   'Human occupancy and resource use',
   'GHG Emissions and Climate Change – Assessment of Upstream GHG Emissions',
   'Soil and soil productivity',
   'Physical and meteorological environment',
   'Vegetation',
   'Fish and fish habitat',
   'Wetlands',
   'Employment and economy',
   'Human health and aesthetics',
   'Wildlife and wildlife habitat',
   'Greenhouse gas (GHG) emissions and climate change',
   'Acoustic environment',
   'Air emissions',
   'Navigation and navigation safety',
   'Electromagnetism and Corona Discharge'],
  'scores': [0.7399547696113586,
   0.05953020229935646,
   0.01749674417078495,
   0.01284861657768488,
   0.005223275627940893,
   0.003144

In [17]:
# Since we have a list of lists for the outputs (because inference was done in batches), we need to flatten the lists together into one.
flatOutput = [item for elem in output for item in elem]

In [18]:
pickled_dataset = 'zero_shot_vcs_train'

In [19]:
import pickle 
with open(f'{pickled_dataset}.pkl', 'wb') as f:
  pickle.dump(flatOutput, f)

In [20]:
import pickle
infile = open(f'{pickled_dataset}.pkl', 'rb')
zs_vcs_dict_list = pickle.load(infile)
infile.close()

In [21]:
zs_vcs_dict_list[0]

{'labels': ['Water quality and quantity',
  'Traditional land and resource use',
  'Environmental Obligations',
  'Rights of Indigenous Peoples',
  'Heritage resources',
  'Social and cultural well-being',
  'Infrastructure and services',
  'Species at Risk or Species of Special Status and related habitat',
  'Human occupancy and resource use',
  'GHG Emissions and Climate Change – Assessment of Upstream GHG Emissions',
  'Soil and soil productivity',
  'Physical and meteorological environment',
  'Vegetation',
  'Fish and fish habitat',
  'Wetlands',
  'Employment and economy',
  'Human health and aesthetics',
  'Wildlife and wildlife habitat',
  'Greenhouse gas (GHG) emissions and climate change',
  'Acoustic environment',
  'Air emissions',
  'Navigation and navigation safety',
  'Electromagnetism and Corona Discharge'],
 'scores': [0.7399547696113586,
  0.05953020229935646,
  0.01749674417078495,
  0.01284861657768488,
  0.005223275627940893,
  0.0031445538625121117,
  0.0027075377

In [22]:
len(zs_vcs_dict_list)

28892

In [23]:
for i in range(0, len(zs_vcs_dict_list)):
  zs_vcs_dict_list[0]['scores']

In [24]:
df_output = pd.DataFrame(output)
df_output

,0,1
0,{'sequence': 'TABLE 3 SUMMARY OF AQUATICS FIEL...,{'sequence': 'TABLE 4 SUMMARY OF WATERCOURSE C...
1,{'sequence': 'TABLE 4 SUMMARY OF WATERCOURSE C...,{'sequence': 'TABLE 4 SUMMARY OF WATERCOURSE C...
2,{'sequence': 'TABLE 4 SUMMARY OF WATERCOURSE C...,{'sequence': 'TABLE 4 SUMMARY OF WATERCOURSE C...
3,{'sequence': 'TABLE 4 SUMMARY OF WATERCOURSE C...,{'sequence': 'TABLE 4 SUMMARY OF WATERCOURSE C...
4,{'sequence': 'TABLE 5 SUMMARY OF WATER QUALITY...,{'sequence': 'TABLE 5 SUMMARY OF WATER QUALITY...
...,...,...
14441,{'sequence': 'TABLE A3 WATER WELLS WITHIN 1 KM...,{'sequence': 'TABLE 1 SUMMARY OF KEY ENVIRONME...
14442,{'sequence': 'TABLE 1 SUMMARY OF KEY ENVIRONME...,{'sequence': 'TABLE 2 POTENTIAL ENVIRONMENTAL ...
14443,{'sequence': 'TABLE 1 GENERAL LAND USE AND ENV...,{'sequence': 'TABLE 2 OCCURRENCES OF SPECIES W...
14444,{'sequence': 'TABLE 2 OCCURRENCES OF SPECIES W...,{'sequence': 'TABLE 2 OCCURRENCES OF SPECIES W...


In [25]:
df_output[df_output.scores > 0.6]

AttributeError: ignored